[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Transcripts4All/tools4all/blob/main/faster-whisper.ipynb)

# Step 1:
# Execute the following task and upload an audio file, or files, to the content directory while you wait for the task to complete.

In [ ]:
!pip install faster-whisper

# Step 2:
# Once the above task has completed and all audio files have successfully been uploaded to the content directory, execute the follow task.

In [ ]:
import glob, os
from faster_whisper import WhisperModel
model = WhisperModel('turbo', device = 'cuda', compute_type = 'float16')
audioFiles = glob.glob('/content/*.*')
for i in range(len(audioFiles)):
  for audioFile in glob.glob(audioFiles[i]):
    lineNumber = 1
    baseFile = os.path.splitext(audioFile)[0]
    with open(baseFile+'.srt', 'w') as srtFile:
      with open(baseFile+'.txt', 'w') as txtFile:
        segments, info = model.transcribe(audioFile, beam_size=5)
        for segment in segments:
          sm, ss = divmod(segment.start, 60)
          sh, sm = divmod(sm, 60)
          em, es = divmod(segment.end, 60)
          eh, em = divmod(em, 60)
          ssi = int(ss)
          ss = ss-ssi
          esi = int(es)
          es = es-esi
          timeStamp = f'{sh:02.0f}:{sm:02.0f}:{ssi:02d},{ss:0.03f}'+' --> '+f'{eh:02.0f}:{em:02.0f}:{esi:02d},{es:0.03f}'
          timeStamp = timeStamp.replace(',0.', ',')
          print('['+timeStamp+']'+segment.text)
          srtFile.write(str(lineNumber)+'\n'+timeStamp+'\n'+segment.text.strip()+'\n\n')
          txtFile.write(segment.text)
          lineNumber = lineNumber+1
    os.remove(audioFile)
    !zip "$baseFile".zip "$baseFile".srt "$baseFile".txt
    !rm "$baseFile".srt "$baseFile".txt

# Step 3:
# Download your ZIP files from the content directory.